In [9]:
import cirq
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import qrcode
import os
from PIL import Image
import pyzbar.pyzbar as pyzbar

# Function to generate QR code
def generate_qr(data, title="QR Code"):
    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=10, border=4)
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill='black', back_color='white')
    img.show(title=title)
    return img

# Function to read QR code
def read_qr(image_path):
    img = Image.open(image_path)
    decoded_objects = pyzbar.decode(img)
    for obj in decoded_objects:
        return obj.data.decode('utf-8')
    return None

# Encryption function for QR Code data
def encrypt_data(key, data):
    cipher = AES.new(key, AES.MODE_ECB)
    padded_data = pad(data.encode('utf-8'), 16)
    ciphertext = cipher.encrypt(padded_data)
    return ciphertext

# Decryption function for AES data
def decrypt_data(key, ciphertext):
    cipher = AES.new(key, AES.MODE_ECB)
    decrypted_data = unpad(cipher.decrypt(ciphertext), 16)
    return decrypted_data.decode('utf-8')

# Double AES Oracle definition
def double_aes_oracle(key_128: bytes, key_128_layer2: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    cipher_128 = AES.new(key_128, AES.MODE_ECB)
    intermediate_ciphertext = cipher_128.encrypt(pad(plaintext, 16))

    cipher_128_layer2 = AES.new(key_128_layer2, AES.MODE_ECB)
    final_ciphertext = cipher_128_layer2.encrypt(intermediate_ciphertext)

    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))
    return ops

# Grover's search setup with double AES
def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 128 + 128
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()
    circuit.append(cirq.H(q) for q in qubits)

    key_128 = os.urandom(16)
    key_128_layer2 = os.urandom(16)
    circuit.append(double_aes_oracle(key_128, key_128_layer2, plaintext, ciphertext, qubits))

    for _ in range(num_iterations):
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Main process
plaintext_message = "This is a secret message"
print("Plaintext Message:", plaintext_message)

# Step 1: Create and display QR code of the plaintext
plaintext_qr = generate_qr(plaintext_message, title="Plaintext QR Code")
plaintext_qr.save("plaintext_qr.png")

# Step 2: Encrypt the plaintext message
key_128 = get_random_bytes(16)
ciphertext = encrypt_data(key_128, plaintext_message)
print("Encrypted Ciphertext:", ciphertext.hex())

# Step 3: Generate QR code for the encrypted ciphertext and display it
ciphertext_qr = generate_qr(ciphertext.hex(), title="Ciphertext QR Code")
ciphertext_qr.save("ciphertext_qr.png")

# Step 4: Read plaintext and ciphertext from QR Codes
plaintext_from_qr = read_qr("plaintext_qr.png")
ciphertext_from_qr = bytes.fromhex(read_qr("ciphertext_qr.png"))

print("Plaintext from QR:", plaintext_from_qr)
print("Ciphertext from QR:", ciphertext_from_qr.hex())

# Step 5: Prepare to run Grover's search algorithm to find the key
key_128_layer2 = get_random_bytes(16)  # Second AES layer key
cipher_128 = AES.new(key_128, AES.MODE_ECB)
intermediate_ciphertext = cipher_128.encrypt(pad(plaintext_from_qr.encode('utf-8'), 16))
cipher_128_layer2 = AES.new(key_128_layer2, AES.MODE_ECB)
final_ciphertext = cipher_128_layer2.encrypt(intermediate_ciphertext)

# Run Grover's search for the double AES-encrypted data
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext_from_qr.encode('utf-8'), final_ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("Qubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


Plaintext Message: This is a secret message
Encrypted Ciphertext: 4867043a2b93ec42d467cde98d33437398ee18c35601db457f432176973611a7
Plaintext from QR: This is a secret message
Ciphertext from QR: 4867043a2b93ec42d467cde98d33437398ee18c35601db457f432176973611a7
Measured Key: q0=0
q1=0
q10=0
q100=0
q101=1
q102=0
q103=1
q104=1
q105=0
q106=0
q107=0
q108=1
q109=0
q11=1
q110=1
q111=1
q112=0
q113=1
q114=0
q115=0
q116=1
q117=0
q118=0
q119=1
q12=0
q120=1
q121=0
q122=1
q123=0
q124=0
q125=0
q126=1
q127=1
q128=1
q129=1
q13=1
q130=0
q131=1
q132=1
q133=0
q134=0
q135=0
q136=0
q137=0
q138=0
q139=0
q14=0
q140=0
q141=1
q142=1
q143=0
q144=0
q145=0
q146=0
q147=0
q148=0
q149=0
q15=1
q150=0
q151=1
q152=1
q153=1
q154=1
q155=0
q156=1
q157=0
q158=0
q159=0
q16=0
q160=1
q161=1
q162=1
q163=0
q164=1
q165=1
q166=0
q167=0
q168=0
q169=0
q17=0
q170=1
q171=1
q172=0
q173=1
q174=1
q175=0
q176=0
q177=1
q178=1
q179=0
q18=0
q180=0
q181=1
q182=0
q183=1
q184=1
q185=1
q186=0
q187=1
q188=1
q189=1
q19=1
q190=0
q191=1
q192=0
q193=

In [2]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the Double AES oracle
def double_aes_oracle(key_128: bytes, key_192: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    # Apply AES-128 encryption
    cipher_128 = AES.new(key_128, AES.MODE_ECB)
    intermediate_ciphertext = cipher_128.encrypt(plaintext)

    # Apply AES-192 encryption
    cipher_192 = AES.new(key_192, AES.MODE_ECB)
    final_ciphertext = cipher_192.encrypt(intermediate_ciphertext)

    # Create a list of operations to mark the solution based on the final ciphertext
    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 128 + 192  # Adjusted qubits for AES-128 + AES-192
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()

    # Initialize the circuit (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # Add the double AES oracle
    key_128 = os.urandom(16)  # Randomly generated AES-128 key for simulation
    key_192 = os.urandom(24)  # Randomly generated AES-192 key for simulation
    circuit.append(double_aes_oracle(key_128, key_192, plaintext, ciphertext, qubits))

    # Apply Grover's diffusion operator
    for _ in range(num_iterations):
        # Inversion about the average
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    # Measure the result
    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Example parameters
plaintext = os.urandom(16)  # Random 16-byte plaintext
key_128 = os.urandom(16)    # AES-128 key
key_192 = os.urandom(24)    # AES-192 key

# Generate the expected ciphertext for the double AES
cipher_128 = AES.new(key_128, AES.MODE_ECB)
intermediate_ciphertext = cipher_128.encrypt(plaintext)
cipher_192 = AES.new(key_192, AES.MODE_ECB)
ciphertext = cipher_192.encrypt(intermediate_ciphertext)

# Run Grover's search for double AES
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("\nQubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


Measured Key: q0=0
q1=0
q10=1
q100=1
q101=0
q102=1
q103=0
q104=0
q105=1
q106=1
q107=0
q108=1
q109=1
q11=1
q110=1
q111=0
q112=0
q113=0
q114=0
q115=0
q116=0
q117=1
q118=1
q119=1
q12=1
q120=0
q121=1
q122=1
q123=1
q124=1
q125=1
q126=0
q127=1
q128=1
q129=1
q13=1
q130=1
q131=0
q132=0
q133=0
q134=0
q135=0
q136=1
q137=1
q138=0
q139=1
q14=0
q140=0
q141=0
q142=0
q143=0
q144=0
q145=1
q146=0
q147=1
q148=1
q149=0
q15=1
q150=1
q151=1
q152=0
q153=1
q154=0
q155=1
q156=1
q157=1
q158=0
q159=1
q16=1
q160=0
q161=1
q162=0
q163=0
q164=0
q165=1
q166=0
q167=0
q168=0
q169=1
q17=0
q170=0
q171=0
q172=0
q173=0
q174=1
q175=1
q176=0
q177=0
q178=1
q179=0
q18=1
q180=1
q181=0
q182=0
q183=1
q184=0
q185=0
q186=1
q187=0
q188=0
q189=0
q19=1
q190=1
q191=1
q192=1
q193=0
q194=0
q195=1
q196=0
q197=0
q198=1
q199=1
q2=1
q20=1
q200=1
q201=0
q202=1
q203=1
q204=1
q205=1
q206=1
q207=1
q208=1
q209=1
q21=1
q210=1
q211=0
q212=1
q213=1
q214=0
q215=1
q216=0
q217=1
q218=0
q219=1
q22=0
q220=1
q221=1
q222=1
q223=1
q224=1
q225=0
q226=0
q227

In [3]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the Double AES oracle
def double_aes_oracle(key_128: bytes, key_256: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    # Apply AES-128 encryption
    cipher_128 = AES.new(key_128, AES.MODE_ECB)
    intermediate_ciphertext = cipher_128.encrypt(plaintext)

    # Apply AES-256 encryption
    cipher_256 = AES.new(key_256, AES.MODE_ECB)
    final_ciphertext = cipher_256.encrypt(intermediate_ciphertext)

    # Create a list of operations to mark the solution based on the final ciphertext
    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 128 + 256  # Adjusted qubits for AES-128 + AES-256
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()

    # Initialize the circuit (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # Add the double AES oracle
    key_128 = os.urandom(16)  # Randomly generated AES-128 key for simulation
    key_256 = os.urandom(32)  # Randomly generated AES-256 key for simulation
    circuit.append(double_aes_oracle(key_128, key_256, plaintext, ciphertext, qubits))

    # Apply Grover's diffusion operator
    for _ in range(num_iterations):
        # Inversion about the average
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    # Measure the result
    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Example parameters
plaintext = os.urandom(16)  # Random 16-byte plaintext
key_128 = os.urandom(16)    # AES-128 key
key_256 = os.urandom(32)    # AES-256 key

# Generate the expected ciphertext for the double AES
cipher_128 = AES.new(key_128, AES.MODE_ECB)
intermediate_ciphertext = cipher_128.encrypt(plaintext)
cipher_256 = AES.new(key_256, AES.MODE_ECB)
ciphertext = cipher_256.encrypt(intermediate_ciphertext)

# Run Grover's search for double AES
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("Qubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


Measured Key: q0=0
q1=1
q10=0
q100=0
q101=0
q102=0
q103=1
q104=1
q105=0
q106=0
q107=0
q108=1
q109=0
q11=0
q110=1
q111=0
q112=0
q113=1
q114=0
q115=1
q116=1
q117=0
q118=1
q119=0
q12=0
q120=1
q121=1
q122=1
q123=0
q124=0
q125=1
q126=1
q127=0
q128=0
q129=1
q13=0
q130=1
q131=0
q132=0
q133=1
q134=0
q135=0
q136=0
q137=1
q138=0
q139=1
q14=1
q140=1
q141=1
q142=0
q143=1
q144=0
q145=0
q146=1
q147=1
q148=1
q149=0
q15=0
q150=1
q151=1
q152=0
q153=1
q154=1
q155=0
q156=1
q157=0
q158=0
q159=1
q16=0
q160=0
q161=0
q162=1
q163=1
q164=1
q165=1
q166=0
q167=0
q168=0
q169=1
q17=0
q170=0
q171=1
q172=1
q173=0
q174=0
q175=0
q176=1
q177=0
q178=0
q179=0
q18=0
q180=1
q181=0
q182=1
q183=1
q184=0
q185=0
q186=1
q187=0
q188=0
q189=1
q19=1
q190=1
q191=0
q192=1
q193=0
q194=0
q195=0
q196=1
q197=1
q198=0
q199=0
q2=0
q20=1
q200=0
q201=0
q202=1
q203=0
q204=1
q205=0
q206=0
q207=0
q208=0
q209=1
q21=0
q210=1
q211=0
q212=0
q213=0
q214=1
q215=0
q216=0
q217=0
q218=0
q219=0
q22=0
q220=0
q221=1
q222=0
q223=1
q224=0
q225=0
q226=1
q227

In [4]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the Double AES oracle
def double_aes_oracle(key_192: bytes, key_192_layer2: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    # Apply AES-192 encryption
    cipher_192 = AES.new(key_192, AES.MODE_ECB)
    intermediate_ciphertext = cipher_192.encrypt(plaintext)

    # Apply AES-192_layer2 encryption
    cipher_192_layer2 = AES.new(key_192_layer2, AES.MODE_ECB)
    final_ciphertext = cipher_192_layer2.encrypt(intermediate_ciphertext)

    # Create a list of operations to mark the solution based on the final ciphertext
    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 192 + 192  # Adjusted qubits for AES-192 + AES-192_layer2
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()

    # Initialize the circuit (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # Add the double AES oracle
    key_192 = os.urandom(24)  # Randomly generated AES-192 key for simulation
    key_192_layer2 = os.urandom(24)  # Randomly generated AES-192_layer2 key for simulation
    circuit.append(double_aes_oracle(key_192, key_192_layer2, plaintext, ciphertext, qubits))

    # Apply Grover's diffusion operator
    for _ in range(num_iterations):
        # Inversion about the average
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    # Measure the result
    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Example parameters
plaintext = os.urandom(16)  # Random 16-byte plaintext
key_192 = os.urandom(24)    # AES-192 key
key_192_layer2 = os.urandom(24)    # AES-192_layer2 key

# Generate the expected ciphertext for the double AES
cipher_192 = AES.new(key_192, AES.MODE_ECB)
intermediate_ciphertext = cipher_192.encrypt(plaintext)
cipher_192_layer2 = AES.new(key_192_layer2, AES.MODE_ECB)
ciphertext = cipher_192_layer2.encrypt(intermediate_ciphertext)

# Run Grover's search for double AES
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("Qubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


Measured Key: q0=0
q1=1
q10=0
q100=1
q101=0
q102=0
q103=0
q104=1
q105=0
q106=1
q107=1
q108=1
q109=0
q11=1
q110=1
q111=0
q112=0
q113=0
q114=0
q115=1
q116=1
q117=0
q118=1
q119=1
q12=1
q120=1
q121=0
q122=0
q123=1
q124=1
q125=0
q126=0
q127=0
q128=1
q129=0
q13=0
q130=1
q131=0
q132=1
q133=1
q134=0
q135=1
q136=0
q137=0
q138=1
q139=1
q14=1
q140=1
q141=1
q142=1
q143=0
q144=0
q145=1
q146=0
q147=1
q148=0
q149=0
q15=0
q150=0
q151=1
q152=1
q153=1
q154=0
q155=1
q156=0
q157=1
q158=0
q159=1
q16=1
q160=1
q161=0
q162=1
q163=1
q164=1
q165=1
q166=0
q167=0
q168=0
q169=1
q17=1
q170=1
q171=1
q172=0
q173=1
q174=0
q175=0
q176=1
q177=1
q178=0
q179=0
q18=0
q180=0
q181=1
q182=1
q183=0
q184=1
q185=0
q186=0
q187=1
q188=1
q189=1
q19=0
q190=0
q191=1
q192=1
q193=1
q194=0
q195=1
q196=0
q197=1
q198=0
q199=1
q2=0
q20=0
q200=1
q201=1
q202=1
q203=1
q204=1
q205=0
q206=1
q207=1
q208=0
q209=1
q21=1
q210=1
q211=1
q212=1
q213=1
q214=0
q215=1
q216=1
q217=0
q218=0
q219=1
q22=0
q220=1
q221=1
q222=0
q223=1
q224=0
q225=0
q226=1
q227

In [5]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the Double AES oracle
def double_aes_oracle(key_192: bytes, key_256: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    # Apply AES-192 encryption
    cipher_192 = AES.new(key_192, AES.MODE_ECB)
    intermediate_ciphertext = cipher_192.encrypt(plaintext)

    # Apply AES-256 encryption
    cipher_256 = AES.new(key_256, AES.MODE_ECB)
    final_ciphertext = cipher_256.encrypt(intermediate_ciphertext)

    # Create a list of operations to mark the solution based on the final ciphertext
    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 192 + 256  # Adjusted qubits for AES-192 + AES-256
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()

    # Initialize the circuit (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # Add the double AES oracle
    key_192 = os.urandom(24)  # Randomly generated AES-192 key for simulation
    key_256 = os.urandom(32)  # Randomly generated AES-256 key for simulation
    circuit.append(double_aes_oracle(key_192, key_256, plaintext, ciphertext, qubits))

    # Apply Grover's diffusion operator
    for _ in range(num_iterations):
        # Inversion about the average
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    # Measure the result
    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Example parameters
plaintext = os.urandom(16)  # Random 16-byte plaintext
key_192 = os.urandom(24)    # AES-192 key
key_256 = os.urandom(32)    # AES-256 key

# Generate the expected ciphertext for the double AES
cipher_192 = AES.new(key_192, AES.MODE_ECB)
intermediate_ciphertext = cipher_192.encrypt(plaintext)
cipher_256 = AES.new(key_256, AES.MODE_ECB)
ciphertext = cipher_256.encrypt(intermediate_ciphertext)

# Run Grover's search for double AES
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("Qubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


Measured Key: q0=1
q1=0
q10=0
q100=0
q101=1
q102=0
q103=1
q104=1
q105=0
q106=0
q107=0
q108=0
q109=1
q11=0
q110=1
q111=0
q112=1
q113=1
q114=0
q115=0
q116=0
q117=1
q118=0
q119=1
q12=1
q120=1
q121=1
q122=1
q123=0
q124=0
q125=1
q126=1
q127=0
q128=0
q129=0
q13=1
q130=0
q131=0
q132=1
q133=1
q134=0
q135=1
q136=1
q137=1
q138=1
q139=0
q14=1
q140=0
q141=1
q142=0
q143=1
q144=1
q145=0
q146=0
q147=1
q148=1
q149=0
q15=1
q150=0
q151=0
q152=0
q153=1
q154=1
q155=0
q156=1
q157=0
q158=0
q159=0
q16=0
q160=1
q161=1
q162=1
q163=0
q164=0
q165=0
q166=0
q167=1
q168=0
q169=0
q17=0
q170=1
q171=1
q172=0
q173=1
q174=0
q175=1
q176=0
q177=1
q178=1
q179=1
q18=0
q180=0
q181=1
q182=1
q183=1
q184=0
q185=1
q186=0
q187=0
q188=1
q189=0
q19=0
q190=1
q191=0
q192=1
q193=0
q194=1
q195=1
q196=0
q197=0
q198=0
q199=0
q2=0
q20=0
q200=1
q201=0
q202=0
q203=0
q204=1
q205=0
q206=0
q207=1
q208=1
q209=0
q21=0
q210=1
q211=0
q212=1
q213=1
q214=0
q215=1
q216=1
q217=0
q218=0
q219=0
q22=1
q220=1
q221=0
q222=1
q223=1
q224=1
q225=1
q226=1
q227

In [6]:
import cirq
import numpy as np
from Crypto.Cipher import AES
import os

# Define the Double AES oracle
def double_aes_oracle(key_256: bytes, key_256_layer2: bytes, plaintext: bytes, ciphertext: bytes, qubits) -> list:
    # Apply AES-256 encryption
    cipher_256 = AES.new(key_256, AES.MODE_ECB)
    intermediate_ciphertext = cipher_256.encrypt(plaintext)

    # Apply AES-256_layer2 encryption
    cipher_256_layer2 = AES.new(key_256_layer2, AES.MODE_ECB)
    final_ciphertext = cipher_256_layer2.encrypt(intermediate_ciphertext)

    # Create a list of operations to mark the solution based on the final ciphertext
    ops = []
    if final_ciphertext == ciphertext:
        for q in qubits:
            ops.append(cirq.X(q))  # Apply X gate as a marker on matching qubits
    return ops


def grover_search_double_aes(plaintext: bytes, ciphertext: bytes, num_iterations: int):
    num_qubits = 256 + 256  # Adjusted qubits for AES-256 + AES-256_layer2
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]

    circuit = cirq.Circuit()

    # Initialize the circuit (Hadamard on all qubits)
    circuit.append(cirq.H(q) for q in qubits)

    # Add the double AES oracle
    key_256 = os.urandom(32)  # Randomly generated AES-256 key for simulation
    key_256_layer2 = os.urandom(32)  # Randomly generated AES-256_layer2 key for simulation
    circuit.append(double_aes_oracle(key_256, key_256_layer2, plaintext, ciphertext, qubits))

    # Apply Grover's diffusion operator
    for _ in range(num_iterations):
        # Inversion about the average
        circuit.append(cirq.H(q) for q in qubits)
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.Z(qubits[-1])**(len(qubits)-1))
        circuit.append(cirq.X(q) for q in qubits)
        circuit.append(cirq.H(q) for q in qubits)

    # Measure the result
    circuit.append(cirq.measure(q, key=f'q{i}') for i, q in enumerate(qubits))

    return circuit

# Example parameters
plaintext = os.urandom(16)  # Random 16-byte plaintext
key_256 = os.urandom(32)    # AES-256 key
key_256_layer2 = os.urandom(32)    # AES-256_layer2 key

# Generate the expected ciphertext for the double AES
cipher_256 = AES.new(key_256, AES.MODE_ECB)
intermediate_ciphertext = cipher_256.encrypt(plaintext)
cipher_256_layer2 = AES.new(key_256_layer2, AES.MODE_ECB)
ciphertext = cipher_256_layer2.encrypt(intermediate_ciphertext)

# Run Grover's search for double AES
num_iterations = 1  # Adjust based on key space size
circuit = grover_search_double_aes(plaintext, ciphertext, num_iterations)
simulator = cirq.Simulator()
result = simulator.run(circuit)

# Output the measurement result
print("Measured Key:", result)
print("Qubit Count:", len(circuit.all_qubits()))
print("Total Gate Count:", len(list(circuit.all_operations())))


Measured Key: q0=1
q1=0
q10=1
q100=1
q101=0
q102=1
q103=1
q104=1
q105=1
q106=1
q107=0
q108=1
q109=1
q11=0
q110=0
q111=0
q112=0
q113=1
q114=0
q115=0
q116=0
q117=0
q118=0
q119=0
q12=1
q120=1
q121=1
q122=1
q123=0
q124=0
q125=0
q126=1
q127=0
q128=1
q129=1
q13=1
q130=0
q131=0
q132=1
q133=1
q134=1
q135=1
q136=0
q137=0
q138=1
q139=1
q14=1
q140=0
q141=1
q142=1
q143=1
q144=0
q145=0
q146=0
q147=1
q148=0
q149=1
q15=1
q150=1
q151=0
q152=0
q153=0
q154=1
q155=1
q156=0
q157=0
q158=1
q159=1
q16=1
q160=0
q161=0
q162=0
q163=0
q164=1
q165=1
q166=0
q167=0
q168=1
q169=0
q17=0
q170=1
q171=0
q172=1
q173=0
q174=0
q175=0
q176=1
q177=0
q178=1
q179=0
q18=1
q180=0
q181=1
q182=1
q183=0
q184=1
q185=0
q186=0
q187=1
q188=0
q189=1
q19=0
q190=0
q191=1
q192=1
q193=1
q194=1
q195=1
q196=1
q197=1
q198=1
q199=0
q2=0
q20=0
q200=1
q201=0
q202=1
q203=0
q204=1
q205=0
q206=0
q207=0
q208=1
q209=0
q21=0
q210=0
q211=1
q212=0
q213=0
q214=1
q215=0
q216=0
q217=1
q218=0
q219=1
q22=1
q220=1
q221=0
q222=0
q223=1
q224=1
q225=0
q226=0
q227